In [1]:
from shapely.ops import unary_union
import geopandas as gpd
from glob import glob
import folium

### Selezionare i tile che corrispondono all'area di interesse. Francia del sud.

In [2]:
yrange = (41.282893, 45.845442)
xrange = (-1.888275, 10.538767)

In [5]:
files = glob('../../data/Sentinel-2-Shapefile-Index/*.shp')

In [6]:
gdf = gpd.read_file(files[1]).to_crs('epsg:4326')

In [7]:
gdf['centroid_x'] = gdf.geometry.apply(lambda x: x.centroid.x)
gdf['centroid_y'] = gdf.geometry.apply(lambda x: x.centroid.y)

In [8]:
selection = (gdf.centroid_x > xrange[0]) & (gdf.centroid_x < xrange[1]) & \
            (gdf.centroid_y > yrange[0]) & (gdf.centroid_y < yrange[1])

In [9]:
gdf = gdf[selection]

In [10]:
gdf.head()

,Name,geometry,centroid_x,centroid_y
28505,30TXM,"POLYGON Z ((-1.78400 42.44624 0.00000, -0.4497...",-1.131336,41.942375
28506,30TXN,"POLYGON Z ((-1.76619 43.34620 0.00000, -0.4124...",-1.104440,42.842113
28507,30TXP,"POLYGON Z ((-1.74752 44.24655 0.00000, -0.3733...",-1.076262,43.742233
28508,30TXQ,"POLYGON Z ((-1.72796 45.14675 0.00000, -0.3323...",-1.046746,44.642196
28509,30TXR,"POLYGON Z ((-1.70747 46.04627 0.00000, -0.2894...",-1.015834,45.541460


In [11]:
_map_ = folium.Map(location=[44, 5], zoom_start=6, tiles='Stamen Terrain')
style1 = {'fillColor': 'none', 'color': '#dd3497'}
folium.GeoJson(gdf, style_function=lambda x:style1).add_to(_map_)
display(_map_)

In [12]:
boundary = gpd.GeoSeries(unary_union(gdf.geometry.to_list()))
boundary.crs = 'epsg:4326'

In [13]:
_map_ = folium.Map(location=[44, 5], zoom_start=6, tiles='Stamen Terrain')
style1 = {'fillColor': 'none', 'color': '#dd3497'}
folium.GeoJson(boundary, style_function=lambda x:style1).add_to(_map_)
display(_map_)

### Selezionare le Parcelle all'intrno dell'area di interesse con dimensione maggiore di 0.25 ettari.

In [14]:
from tqdm import tqdm
from glob import glob
import pandas as pd

In [15]:
lower_area_bound = (50.*50.) / 10000.

In [ ]:
parcels = pd.concat([gpd.read_file(f) for f in tqdm(glob('../../data/RPGs/*.geojson'))])

 56%|██████████████████████████████████████████████▏                                   | 9/16 [12:11<11:26, 98.11s/it]

In [ ]:
def check_intersect(x, y):
    try:
        return x.intersects(y)
    except:
        return False

In [ ]:
select1 = parcels.geometry.apply(lambda x: check_intersect(x, boundary.iloc[0]))
select2 = parcels['SURF_PARC'] > lower_area_bound

In [ ]:
_map_ = folium.Map(location=[44, 5], zoom_start=6, tiles='Stamen Terrain')
style1 = {'fillColor': 'none', 'color': '#dd3497'}
folium.GeoJson(parcels[select1 & select2].sample(10000), 
               style_function=lambda x:style1).add_to(_map_)
display(_map_)

In [ ]:
parcels[select1 & select2].to_file('../../data/Parcels/Parcels_South_France.geojson', driver='GeoJSON')

### Associazione Parcella-Tile

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

import matplotlib.pyplot as plt
import geopandas as gpd

from tqdm import tqdm
tqdm.pandas()

In [ ]:
parcels = gpd.read_file('../../data/Parcels/Parcels_South_France.geojson')

In [ ]:
parcels['tile'] = parcels.geometry.parallel_apply(lambda y: (gdf[
                    gdf.geometry.apply(lambda x: check_intersect(x, y))
                        ].Name.tolist()))
parcels = parcels.explode('tile')

### Analisi distribuzione Crops per Tile

Ordinare i tile in modo da avere:

- il maggior numero di parcelle per tile
- il miglior bilanciamento tra le macro-classi vegetative 

In [ ]:
avoid_labels = {'SNE': 'Terreno Inutilizzato',

                'PPH': 'Prato Permanente',

                'SPH': 'Superficie pastorale',
                'SPL': 'Superficie pastorale',
                
                'J6P': 'Maggese - rotazione lunga',
                'J6S': 'Maggese - rotazione media',
                'J5M': 'Maggese - temporaneo',

                'PRL': 'Prato - rotazione lunga',
                'PTR': 'Prato - temporaneo',

                'LUZ': 'Erba Medica', 
                
                'BOP': 'Bosco',
                'TRE': 'Altro Trifoglio',
                'BOR': 'Confine del campo',
               
               }


valid_labels = {'VRC': 'Vitigno - Produttivo',
                'VRN': 'Vitigno - Non  Produttivo',

                'BTH': 'Grano tenero invernale',
                'BDH': 'Grano duro invernale',

                'MIS': 'Mais',
                'TRN': 'Girasole',
                
                'ORH': 'Orzo Invernale', 
                'ORP': 'Orzo Primaverile',

                'CZH': 'Colza Invernale',
                
                'MIE': 'Insilato di mais',
                'TTH': 'Triticale Invernale',

                'RGA': 'Loglio - temporaneo',
                'MLG': 'Miscela di legumi',
                'MCR': 'Cereali Misti', 
                
                'VRG': 'Frutteto',
                'TRN': 'Girasole',
 
                'SOG': 'Sorgo',
                'SOJ': 'Soia',
                'NOX': 'Noce',
                'SGH': 'Segale Invernale',
                
                'MLF': 'Misto di Legumi da Foraggio',
                'BTA': 'Fascia Tampone' }

In [ ]:
parcels = parcels[parcels.CODE_CULTU.isin(list(avoid_labels.keys()))==False]

In [ ]:
parcel_counter = parcels.groupby('tile').count()['ID_PARCEL'].sort_values(ascending=False)

In [ ]:
parcels = parcels[parcels.tile.isin(parcel_counter[parcel_counter > 1.e4].index)]

In [ ]:
# Selezioniamo le specie vegetative che hanno almento 1000 rappresentanti in media per tile 
crops_distr = parcels.groupby(by=['tile', 'CODE_CULTU']).ID_PARCEL.count().unstack(level=1).fillna(0)
selected = crops_distr.describe().T.sort_values(by='mean', ascending=False).iloc[0:22]

In [ ]:
selected

In [ ]:
parcels = parcels[parcels.CODE_CULTU.isin(selected.index)]

In [ ]:
parcels.groupby(by=['tile', 'CODE_CULTU']).ID_PARCEL.count().unstack(level=1).fillna(0)

In [ ]:
to_plot = parcels.groupby(by=['tile', 'CODE_CULTU']).ID_PARCEL.count().unstack(level=1).fillna(0)
to_plot = to_plot[selected.index].loc[to_plot.sum(axis=1).sort_values(ascending=False).index]
to_plot.plot.bar(alpha=.75, figsize=(15, 5), stacked=True, legend=True)
plt.legend(loc='upper center', ncol=12, fancybox=True, shadow=False, bbox_to_anchor=(0.5, 1.2))
plt.tight_layout()

In [ ]:
import pandas as pd

In [ ]:
veg = pd.read_csv('../../data/Vegetation-Dictionary/REF_CULTURES_GROUPES_CULTURES_2020.csv', sep=';')[['CODE_CULTURE','LIBELLE_CULTURE']]
veg = veg.set_index('CODE_CULTURE').LIBELLE_CULTURE

In [ ]:
veg[to_plot.columns[(to_plot.columns.isin(list(valid_labels.keys()))==False) & 
                    (to_plot.columns.isin(list(avoid_labels.keys()))==False)]]

### Visualization of the selected parcels within the parcell

In [ ]:
selected_tiles = list(to_plot.index[:3])
print(selected_tiles)

In [ ]:
_map_ = folium.Map(location=[44, 5], zoom_start=6, tiles='Stamen Terrain')
style1 = {'fillColor': 'none', 'color': '#dd3497'}
folium.GeoJson(parcels[parcels.tile.isin(selected_tiles)].sample(10000), 
               style_function=lambda x:style1).add_to(_map_)
display(_map_)